In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn_extra.cluster import KMedoids
from skimage import color
from collections import Counter
import matplotlib.gridspec as gridspec

# Step 1: Load the image
imagePath = "myenv/img/cameronmarygold/2023-10-13_22-52-59_CyW0A9fPySK/387799770_18392162299048990_5074269383753542983_n.jpg"
image = Image.open(imagePath[6:])
image_np = np.array(image)

# Ensure the image is in RGB format
if image.mode != 'RGB':
    image = image.convert('RGB')
    image_np = np.array(image)

# Remove alpha channel if present
if image_np.shape[2] == 4:
    image_np = image_np[:, :, :3]

# Step 2: Normalize and convert to LAB color space
image_normalized = image_np / 255.0
image_lab = color.rgb2lab(image_normalized)

# Step 3: Reshape image data for clustering
pixels_lab = image_lab.reshape(-1, 3)

# Step 4: Perform k-medoids clustering with k=8
k = 8  # Set the number of clusters
kmedoids = KMedoids(n_clusters=k, random_state=42)
kmedoids.fit(pixels_lab)

# Step 5: Process cluster centers
cluster_centers_lab = kmedoids.cluster_centers_
cluster_centers_rgb = color.lab2rgb(cluster_centers_lab.reshape(1, -1, 3))
cluster_centers_rgb = np.squeeze(cluster_centers_rgb)
cluster_centers_rgb_uint8 = np.clip(cluster_centers_rgb * 255, 0, 255).astype(int)

# Step 6: Quantify and visualize cluster sizes
labels = kmedoids.labels_
counts = Counter(labels)
total_pixels = sum(counts.values())

# Sort clusters by the number of pixels
sorted_counts = counts.most_common()
sorted_cluster_indices = [item[0] for item in sorted_counts]
sorted_cluster_sizes = [item[1] for item in sorted_counts]
sorted_cluster_percentages = [(count / total_pixels) * 100 for count in sorted_cluster_sizes]
sorted_colors_rgb = np.array([cluster_centers_rgb_uint8[i] for i in sorted_cluster_indices])

# Prepare colors for bar charts (normalized to [0, 1])
bar_colors = sorted_colors_rgb / 255
cluster_numbers = range(1, k + 1)

# Create a figure with GridSpec
fig = plt.figure(figsize=(12, 8))
gs = gridspec.GridSpec(nrows=2, ncols=2, width_ratios=[3, 1], height_ratios=[4, 1])

# Display the original image in the left column, top row
ax_image = fig.add_subplot(gs[0, 0])
ax_image.imshow(image_np)
ax_image.axis('off')
ax_image.set_title('Original Image')

# Plot color swatches underneath the image in the left column, bottom row
gs_swatches = gridspec.GridSpecFromSubplotSpec(1, k, subplot_spec=gs[1, 0])
for i in range(k):
    ax = fig.add_subplot(gs_swatches[0, i])
    swatch = np.zeros((50, 50, 3), dtype=int)
    swatch[:, :] = sorted_colors_rgb[i]
    ax.imshow(swatch)
    ax.axis('off')
    hex_color = '#{:02x}{:02x}{:02x}'.format(*sorted_colors_rgb[i])
    percentage = sorted_cluster_percentages[i]
    ax.set_title(f"{hex_color}\n{percentage:.1f}%", fontsize=8)

# Plot the bar chart in the right column, spanning both rows
ax_bar = fig.add_subplot(gs[:, 1])
ax_bar.bar(cluster_numbers, sorted_cluster_percentages, color=bar_colors)
ax_bar.set_xlabel('Cluster')
ax_bar.set_ylabel('Percentage of Pixels')
ax_bar.set_title('Percentage of Pixels per Cluster')
ax_bar.set_xticks(cluster_numbers)

plt.tight_layout()
plt.show()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/greyson/Projects/custom_gallery/myenv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/greyson/Projects/custom_gallery/myenv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/greyson/Projects/custom_gallery/myenv/lib/python3.12/site-packages/ipykernel/kernelapp.py

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).